# Modified Gale Shapley Algorithm

In [13]:
import pandas as pd

In [14]:
syndata = pd.read_csv('synData9.csv')
realdata = pd.read_csv('kag_risk_factors_cervical_cancer.csv')

In [15]:
syndata.iloc[0] #Use this to access a row
syndata.loc[0, :] #Use this to access a row
x = syndata.loc[:,"Age"] #Use this to access a column

In [16]:
syndata.columns[1]

'Number of sexual partners'

Cdoe for finding the mean. Removes any unknown values

In [32]:
def mean_for_column(data1, column):
    """
    Finds the mean for a specific column
    """
    tot = 0 
    for i in range(len(data1)):
        #print(data1.loc[i, column])
        if data1.loc[i, column] == "?" or data1.loc[i, column] == -1:
            pass
        else:
            tot = tot + float(data1.loc[i, column])
    tot = tot/(len(data1))
    return tot
    
def mean_for_cols(data1):
    """
    Computes the mean for all the columns 
    """
    ar = []
    for col in data1.columns: 
        ar.append(mean_for_column(data1, col))
    return ar

In [33]:
syndata_means = mean_for_cols(syndata)
realdata_means = mean_for_cols(realdata)

Data Cleaning

In [8]:
def clean(data1):
    """
    Replaces the missing data with a -1 value 
    """
    for i in range(len(data1)):
        for f in data1.columns:
            #print(data1.loc[i, f])
            if data1.loc[i, f] == "?":
                data1.loc[i, f] = -1
                break
    data1 =data1.set_axis([f for f in range(len(data1))], axis='index')
    return data1

realdata = clean(realdata)
syndata = clean(syndata)



In [9]:
realdata

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,?,0,0,0,0,0,0,0,0
2,34,1.0,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,-1,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,-1,?,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,34,3.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,?,0,0,0,0,0,0,0,0
854,32,2.0,19.0,1.0,0.0,0.0,0.0,1.0,8.0,0.0,...,-1,?,0,0,0,0,0,0,0,0
855,25,2.0,17.0,0.0,0.0,0.0,0.0,1.0,0.08,0.0,...,-1,?,0,0,0,0,0,0,1,0
856,33,2.0,24.0,2.0,0.0,0.0,0.0,1.0,0.08,0.0,...,-1,?,0,0,0,0,0,0,0,0


Now we work on creating a compare function which is being designed to comapre all the rows of one dataset to one specific row of another dataset

In [10]:
def compare(data1, data2, row_comp):
    """
    This is a comparsion function  
    data1: First dataset for comparsion, 1 set row
    data2: Second dataset for comparsion
    row_comp: The row we want compared
    """
    #Goes for 1 section of the real data and then loops for every row of the synthesized data
    for i in range(len(data2)):
        print('Real data: ' + str(data1.loc[row_comp, 'Age']) + '. Fake data: ' +str(data2.loc[i, 'Age']))
        #Comparsion function to compare each the set real row to current fake row
        similiar(data1, data2, i, row_comp)
        if i==1:
            return

In [11]:
def similiar(data1, data2, ind, row_comp): #, weight):
    """
    Computes how similiar the two datasets are 
    """
    for i in data1.columns:
        print(ind, data1.loc[row_comp,:], data2.loc[ind,:])
        print(i)
        if i ==1:
            return

In [12]:
compare(realdata,syndata, 0)

Real data: 18. Fake data: 12
0 Age                                     18
Number of sexual partners              4.0
First sexual intercourse              15.0
Num of pregnancies                     1.0
Smokes                                 0.0
Smokes (years)                         0.0
Smokes (packs/year)                    0.0
Hormonal Contraceptives                0.0
Hormonal Contraceptives (years)        0.0
IUD                                    0.0
IUD (years)                            0.0
STDs                                   0.0
STDs (number)                          0.0
STDs:condylomatosis                    0.0
STDs:cervical condylomatosis           0.0
STDs:vaginal condylomatosis            0.0
STDs:vulvo-perineal condylomatosis     0.0
STDs:syphilis                          0.0
STDs:pelvic inflammatory disease       0.0
STDs:genital herpes                    0.0
STDs:molluscum contagiosum             0.0
STDs:AIDS                              0.0
STDs:HIV               